In [1]:
from gettext import dpgettext

import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://hadits.site/shahih"
response = requests.get(url)

In [3]:
response

<Response [200]>

In [4]:
response.text

'<!DOCTYPE html>\n<html>\n<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  \n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <title>\n          Hadits.Site | Mencari Hadits Dan Derajatnya\n      </title>\n  <!-- Tell the browser to be responsive to screen width -->\n  <meta name="description" content="Aplikasi hadits.site membantu untuk menelusuri hadits-hadits Nabi dan kesimpulan derajatnya berdasarkan penilaian para ulama">\n  <meta name="keywords" content="aplikasi,hadits,hadis,hadits Nabi,derajat hadits,palsu,dhaif,sahih,shahih,ulama,ahli hadits">\n  <meta name="robots" content="index,follow">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  \n  <meta name="csrf-token" content="PH0hs50EwGrxqQV7Vq6slTXAHMdiIfbOorqbRuK6" />\n  <meta property="og:site_name" content="Hadits.Site" />\n  <meta property="og:type" content="website" />\n      <meta property="og:title" content="Hadits.Site | Mencari Hadits Dan Derajatnya" />\n 

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
cards = soup.find_all("div", class_="card result-item")

In [7]:
cards

[<div class="card result-item">
 <div class="card-header">
 <a href="/hadits/1393"><img height="17" src="https://hadits.site/img/link.png"/></a>
 </div>
 <div class="card-body">
 <div class="row">
 <div class="col-md-12">
 <p class="text-arabic result-text arabic">فِي رِوَايَتِهِ "إِذَا وَلَدَتِ الأَمَةُ بَعْلَهَا" يَعْنِي السَّرَارِيّ</p>
 </div>
 <div class="col-md-12">
 <p class="result-text">“Dalam riwayat lain, "... ketika para al-amatu melahirkan tuannya", al-amatu maksudnya para budak wanita yang digauli.”</p>
 </div>
 </div>
 </div>
 <div class="card-footer">
 <span class="" style="font-size: 0.9em;">
 <span class="text-muted">Perawi:</span> <a href="https://hadits.site/home?q=Abu%2BHurairah">Abu Hurairah</a>
 <span class="text-muted">Ulama hadits:</span> <a href="https://hadits.site/home?q=Muslim%2Bbin%2BHajjaj">Muslim bin Hajjaj</a>
 <span class="text-muted">Nama kitab:</span> <a href="https://hadits.site/home?q=Shahih%2BMuslim">Shahih Muslim</a>
 <span class="text-muted">Nom

In [8]:
card = cards[0]

In [10]:
arabic = card.find("p", class_="text-arabic result-text arabic").text

In [11]:
arabic

'فِي رِوَايَتِهِ "إِذَا وَلَدَتِ الأَمَةُ بَعْلَهَا" يَعْنِي السَّرَارِيّ'

In [14]:
translate_en = card.find(lambda tag: tag.name == "p" and tag.get("class") == ["result-text"]).text

In [15]:
translate_en

'“Dalam riwayat lain, "... ketika para al-amatu melahirkan tuannya", al-amatu maksudnya para budak wanita yang digauli.”'

In [19]:
narator = card.find("span", string="Perawi:").find_next("a").text
ulama_hadits = card.find("span", string="Ulama hadits:").find_next("a").text
khitab_name = card.find("span", string="Nama kitab:").find_next("a").text
number = card.find("span", string="Nomor:").find_next("strong").text
level_of_hadith = card.find("span", string="Derajat hadits:").next_sibling.strip()

In [20]:
narator

'Abu Hurairah'

In [21]:
ulama_hadits

'Muslim bin Hajjaj'

In [29]:
def get_text_or_none(tag, next_tag_selector, attr="text"):
    """Safely extract text from the next tag if found, otherwise return None."""
    if tag:
        next_tag = tag.find_next(next_tag_selector)
        if next_tag:
            return next_tag.get_text(strip=True) if attr == "text" else next_tag[attr]
    return None

def get_data(card):
    arabic = card.find("p", class_="text-arabic result-text arabic").text
    translate_id = card.find(lambda tag: tag.name == "p" and tag.get("class") == ["result-text"]).text
    narator_tag = card.find("span", string="Perawi:")
    narator = get_text_or_none(narator_tag, "a")

    ulama_tag = card.find("span", string="Ulama hadits:")
    hadith_scholar = get_text_or_none(ulama_tag, "a")

    kitab_tag = card.find("span", string="Nama kitab:")
    khitab_name = get_text_or_none(kitab_tag, "a")

    number_tag = card.find("span", string="Nomor:")
    number = get_text_or_none(number_tag, "strong")

    level_tag = card.find("span", string="Derajat hadits:")
    level_of_hadith = level_tag.next_sibling.strip() if level_tag and level_tag.next_sibling else None
    return {
        'arabic': arabic, 'translate_id': translate_id, "narator": narator, "hadith_scholar": hadith_scholar,
        "source": khitab_name, "hadith_no": number, "level_of_hadith": level_of_hadith
    }

In [26]:
import pandas as pd

In [30]:
df = pd.DataFrame(columns=["arabic", "translate_id", "narator", "hadith_scholar", "source", "hadith_no", "level_of_hadith"])

In [32]:
for i in range(1,158):
    req = requests.get(url, params={"page": i})
    soup = BeautifulSoup(req.text, "html.parser")
    cards = soup.find_all("div", class_="card result-item")
    for card in cards:
        data = get_data(card)
        df = df._append(data, ignore_index=True)

In [33]:
df

,arabic,translate_id,narator,hadith_scholar,source,hadith_no,level_of_hadith
0,"فِي رِوَايَتِهِ ""إِذَا وَلَدَتِ الأَمَةُ بَعْل...","“Dalam riwayat lain, ""... ketika para al-amatu...",Abu Hurairah,Muslim bin Hajjaj,Shahih Muslim,6,Shahih
1,عَنْ يَحْيَى بْنِ يَعْمَرَ، عَنِ ابْنِ عُمَرَ،...,"“Dari Yahya bin Ya'mar dari Ibnu Umar, dari Um...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,4,Shahih
2,عن يحيى بن يَعْمَرَ، وَحُمَيْدِ بْنِ عَبْدِ ال...,“Dari Yahya bin Ya'mar dan Humaid bin 'Abdur-R...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,3,Shahih
3,عَنْ يَحْيَى بْنِ يَعْمَرَ قَالَ لَمَّا تَكَلّ...,"“Yahya bin Ya'mar berkata: ""Ketika Ma'bad mula...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,2,Shahih
4,كَانَ أَوَّلَ مَنْ قَالَ فِي الْقَدَرِ بِالْبَ...,“Orang yang pertama kali membahas takdir di Ba...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,1,Shahih
...,...,...,...,...,...,...,...
778,مَن سَمِعَ النِّداءَ فلَم يأتِ فلا صَلاةَ لَه ...,"“Barangsiapa yang mendengar adzan, namun tidak...",Abdullah bin Abbas,Ibnu Hajar Al Asqalani,Bulughul Maram,114,Shahih
779,إنَّ لك من الأجرِ على قدْرِ نَصَبِك و نفقتِك,“Sesungguhnya besar pahala sebanding dengan ke...,Aisyah,Muhammad Nashiruddin Al Albani,Shahih Jami' Ash Shaghir,2160,Shahih
780,يَوَدُّ أَهلُ العَافِيَةِ يَومَ القِيَامَةِ حِ...,“Orang-orang yang dahulu sehat di dunia kelak ...,Jabir bin Abdillah,Muhammad Nashiruddin Al Albani,Shahih At Tirmidzi,2402,Shahih
781,: خطبنا رسول الله صلى الله عليه وسلم، فأقبل ال...,“Rasulullah Shallallahu ‘alaihi wa sallam berk...,Abdullah bin Buraidah,Muhammad Nashiruddin Al Albani,Shahih Abu Daud,1109,Shahih


In [34]:
df.to_csv("csv/hadith.csv", index=False)